In [28]:
import cupy as cp
from cupy import asnumpy 
from cuml.datasets.classification import make_classification
from cuml.preprocessing.model_selection import train_test_split
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.metrics import max_error as sklearn_max_error
import time


# synthetic dataset dimensions
n_samples = 2**15
n_features = 100
n_classes = 4
n_informative = 10

# random forest depth and size
n_estimators = 25
max_depth = 10

# generate synthetic data [ binary classification task ]
X, y = make_classification ( n_classes = n_classes,
                             n_features = n_features,
                             n_samples = n_samples,
                             n_informative = n_informative,
                             random_state = 0 )

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 0 )

model = cuRF( max_depth = max_depth, 
              n_estimators = n_estimators,
              random_state  = 0 )

trained_RF = model.fit ( X_train, y_train )

predictions = model.predict ( X_test )

def max_error(y_true, y_pred):
    y_true = y_true.astype(cp.float32)
    y_pred = y_pred.astype(cp.float32)
    error = cp.ElementwiseKernel(
        'T y_pred, T y_true',
        'T diff',
        'diff = y_pred - y_true',
        'error')
    diff = error(y_pred, y_true)
    return cp.max(diff)
s = time.time()
cu_score = max_error( y_test, predictions )
e = time.time()
print("Time consumed by cuml: ", e-s)
p = time.time()
sk_score = sklearn_max_error( asnumpy( y_test ), asnumpy( predictions ) )
d = time.time()
print("Time consumed by sklearn: ", d-p)

print( " cuml max_error: ", cu_score )
print( " sklearn max_error : ", sk_score )


Time consumed by cuml:  0.0003638267517089844
Time consumed by sklearn:  0.0008661746978759766
 cuml max_error:  3.0
 sklearn max_error :  3.0


<ipython-input-28-1eac379c32d7>:29: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  model = cuRF( max_depth = max_depth,
